In [1]:
import terl
from terl.envs import TradingEnv, make_env
from terl.config import EnvConfigManager, config_checker
from terl.common.utils import random_index
import numpy as np
import pandas as pd
import vaex
from datetime import datetime

from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer


In [2]:
ecm = EnvConfigManager()

In [3]:
conf = ecm.get_config('Env 2')
conf.update({'data_loader':'pd'})
ecm.save_config(conf,'Env 2')

In [2]:
env = make_env('Env 2')

INFO:MainThread:numexpr.utils:NumExpr defaulting to 8 threads.


In [4]:
env._db

{'GBPUSD_15':         GBPUSD_15_tick_volume  GBPUSD_15_high  GBPUSD_15_close
 0                        2781         1.54450          1.53380
 1                        2571         1.53600          1.52250
 2                        2711         1.54150          1.53870
 3                        2921         1.54600          1.53550
 4                        2711         1.53800          1.53650
 ...                       ...             ...              ...
 544029                   2456         1.33649          1.33560
 544030                   2525         1.33832          1.33779
 544031                   4181         1.33902          1.33823
 544032                   4720         1.34078          1.34052
 544033                   2852         1.34063          1.33889
 
 [544034 rows x 3 columns]}

In [4]:
%%timeit
env.reset()

3.04 ms ± 74.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
df = env._db.get("GBPUSD_15")

In [15]:
for col in df.column_names:
    df.rename(col, f"GBPUSD_15_{col}" )

In [19]:
df.columns

['GBPUSD_15_open',
 'GBPUSD_15_high',
 'GBPUSD_15_low',
 'GBPUSD_15_close',
 'GBPUSD_15_tick_volume']

In [3]:
cm = EnvConfigManager()

In [5]:
config_checker(cm.get_config('Test 1'))

In [7]:
%timeit random_index(60, 100000)

357 ns ± 2.49 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
